### Parsing

In [2]:
from bs4 import BeautifulSoup
import urllib2

#### What do you want?

In [30]:
num_pages = 3
articles = []

#### Define the functions

In [272]:
def getSoupFromUrl(url):
    req = urllib2.Request(url, headers={'User-Agent' : "Chrome"}) 
    con = urllib2.urlopen(req)
    page = con.read()
    soup = BeautifulSoup(page, "lxml")
    con.close()
    return soup

In [555]:
base_url = 'http://monitor.icef.com/'
tag="div"
attr1="class"
attr2="article"
articles = []
num_pages = 1

for i in range(num_pages):
    url = 'http://monitor.icef.com/page/{}/'.format(i)
    soup = getSoupFromUrl(url)
    print("..{}".format(i))
    result = soup.find_all(tag, attrs={attr1: attr2})
    for line in result[0:10]:
        for row in line.find_all('a'):
            soup = getSoupFromUrl(row.get("href"))
            paras = soup.find_all("p")[:-14:]
            content = " ".join([para.getText() for para in paras])
            articles.append(content)

..0


### Save article content only

In [ ]:
import codecs
import re

In [ ]:
filename = 'news.txt'
file = codecs.open(filename, "w", "utf-8")
for article in articles:
    file.write(re.sub( '\s+', ' ', article).strip() + '\n')
file.close()

### Extract the additional information you need

In [575]:
# GET TITLES

title = []
for line in result[0:10]:
    for row in line.find_all('a'):
        title.append(row.get_text())

In [576]:
title

[u'Australia: Growth remains strong at mid-year',
 u'Brazilian outbound continues to strengthen with a trend to longer stays abroad',
 u'UK: Net migration questioned again as new data shows 97% of international students leave in time',
 u'Canadian language programmes register modest growth in 2016',
 u'Ireland moves to expand student housing stock',
 u'Slowing growth in foreign secondary enrolment carries important implications for US recruiters',
 u'Indian higher ed enrolment skyrocketing but foreign numbers remain modest',
 u'US: Most university business officers anticipate financial challenges ahead',
 u'The business case for translating your website',
 u'Megatrend: The shift to emerging markets']

In [577]:
# GET LINKS
link = []
for line in result[0:10]:
    for row in line.find_all('a'):
        link.append(row["href"])

In [578]:
link

['http://monitor.icef.com/2017/08/australia-growth-remains-strong-mid-year/',
 'http://monitor.icef.com/2017/08/brazilian-outbound-continues-strengthen-trend-longer-stays-abroad/',
 'http://monitor.icef.com/2017/08/uk-net-migration-questioned-new-data-shows-97-international-students-leave-time/',
 'http://monitor.icef.com/2017/08/canadian-language-programmes-register-modest-growth-2016/',
 'http://monitor.icef.com/2017/08/ireland-moves-expand-student-housing-stock/',
 'http://monitor.icef.com/2017/08/slowing-growth-foreign-secondary-enrolment-carries-implications-us-recruiters/',
 'http://monitor.icef.com/2017/08/indian-higher-ed-enrolment-skyrocketing-foreign-numbers-remain-modest/',
 'http://monitor.icef.com/2017/08/us-university-business-officers-anticipate-financial-challenges-ahead/',
 'http://monitor.icef.com/2017/08/business-case-translating-website/',
 'http://monitor.icef.com/2017/08/megatrend-shift-emerging-markets/']

In [572]:
# GET DATES

dates = []

for line in result[0:10]:
    for row in line.find_all('a'):
        soup = getSoupFromUrl(row.get("href"))
        dates.append(str(soup.find_all("span", attrs={"style": "margin-right:40px;position:relative;top:13px;"})))

In [580]:
date = []

for i in dates:
    date.append(re.search(r'\[\<span style\=\"margin\-right\:40px\;position\:relative\;top\:13px\;\"\>(.*?)\<\/span\>', i).group(1))

In [581]:
date

['30 Aug  2017',
 '29 Aug  2017',
 '28 Aug  2017',
 '23 Aug  2017',
 '23 Aug  2017',
 '16 Aug  2017',
 '16 Aug  2017',
 '9 Aug  2017',
 '8 Aug  2017',
 '2 Aug  2017']

### Save article content and general information

In [585]:
lib = {z[0]:list(z[1:]) for z in zip(title,link,date,articles)}

In [587]:
lib.keys()

[u'The business case for translating your website',
 u'Megatrend: The shift to emerging markets',
 u'Brazilian outbound continues to strengthen with a trend to longer stays abroad',
 u'Slowing growth in foreign secondary enrolment carries important implications for US recruiters',
 u'US: Most university business officers anticipate financial challenges ahead',
 u'Canadian language programmes register modest growth in 2016',
 u'Australia: Growth remains strong at mid-year',
 u'Indian higher ed enrolment skyrocketing but foreign numbers remain modest',
 u'Ireland moves to expand student housing stock',
 u'UK: Net migration questioned again as new data shows 97% of international students leave in time']

### Language processing

#### How to return article information concatenated with the sentence of interest?
        - Concatenate/"tag" every string with its article information (title, date, link)
        - When processing the strings, ignore the additional information or "tag" but include it when printing
        - Perhaps that could be achieved by adding the addtional information to the stopwords

In [602]:
libl = lib.items()

In [606]:
[item[1] for item in libl]

[['http://monitor.icef.com/2017/08/business-case-translating-website/',
  '8 Aug  2017',
  u"Short on time? Here are the highlights: If there were only 100 people in the world, says web globalisation expert John Yunker, more than half of them would live in Asia. North America would account for only seven. This is a bit of a jarring mental exercise but one that highlights how much population and economic growth has taken hold outside of the world\u2019s developed markets. And it\u2019s an idea that underpins the business case for investing more in adapting the structure, design, and content of your website for international markets. The following table illustrates the composition of global Internet usage in terms of the native languages of users, and it offers a couple of broad observations on the state of the web today. \nInternet users by native language, March 2017. Source: Internet World Stats First, only about a quarter of web users are native English speakers. There is an importan

#### Processing the sentences

In [ ]:
from __future__ import division
import nltk, re, pprint
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize

In [ ]:
f = open('news.txt')
raw = f.read()

In [ ]:
def sentence_finder(text,word):
        sentences=sent_tokenize(text)
        return [sent for sent in sentences if word in word_tokenize(sent)]

In [ ]:
digits = []
digits.append(re.findall('\d+', raw))
for i in digits[0]:
    sentences.append(sentence_finder(raw,i))

#### Find sentences that contain xxxx

In [ ]:
sentence_finder(raw,'2016')